# UAT for Cycle 02: The Physics-Informed Generator

This notebook demonstrates the functionality of the `PhysicsInformedGenerator` from Cycle 02. We will walk through configuring the generator, running it to create a diverse set of atomic structures for an alloy, and visualizing the results.

**Note**: Due to a dependency conflict with the `icet` library and Python 3.12, this UAT uses a **mocked** version of the generator. It produces hard-coded structures instead of performing real SQS calculations, but the subsequent strain and rattle transformations are the real implementation.

## Part 1: Configuration

In [ ]:
import os
import sys
from ase.visualize import view

# Add the project root to the Python path
sys.path.append(os.path.abspath(os.path.join('..', '..')))

from mlip_autopipec.config_schemas import (
    SystemConfig,
    DFTConfig,
    DFTInput,
    GeneratorParams,
    AlloyParams,
)
from mlip_autopipec.modules.generator import PhysicsInformedGenerator

print("✓ Imports successful")

In [ ]:
# Step 1.1: Create a SystemConfig for a CuAu alloy
config = SystemConfig(
    dft=DFTConfig(
        input=DFTInput(pseudopotentials={"Cu": "Cu.upf", "Au": "Au.upf"})
    ),
    generator=GeneratorParams(
        alloy_params=AlloyParams(
            strain_magnitudes=[0.95, 1.05], rattle_std_devs=[0.1]
        )
    ),
)

print("✓ SystemConfig for CuAu alloy created successfully.")

## Part 2: Generation

In [ ]:
# Step 2.1: Instantiate the generator
generator = PhysicsInformedGenerator(config)

# Step 2.2: Run the generation process
generated_structures = generator.generate()

# Step 2.3: Verify the number of generated structures
print(f"Total number of structures generated: {len(generated_structures)}")
# Expected: (1 base + 2 strains) * (1 base + 1 rattle) = 3 * 2 = 6
assert len(generated_structures) == 6

## Part 3: Verification and Visualisation

In [ ]:
# Step 3.1: Inspect the base structure
base_structure = generated_structures[0]
print(f"Base structure chemical formula: {base_structure.get_chemical_formula()}")
assert base_structure.get_chemical_formula() == 'Au4Cu4'

In [ ]:
# Visualize the base structure (requires a graphical backend)
# view(base_structure)

In [ ]:
# Step 3.2: Verify the strain application
strained_structure = generated_structures[2] # First strained structure
base_volume = base_structure.get_volume()
strained_volume = strained_structure.get_volume()

print(f"Volume of base structure: {base_volume:.2f} Å³")
print(f"Volume of strained structure: {strained_volume:.2f} Å³")
assert not np.isclose(base_volume, strained_volume)

In [ ]:
# Step 3.3: Verify the rattle application
rattled_structure = generated_structures[1] # Rattled version of the base structure
base_positions = base_structure.get_positions()
rattled_positions = rattled_structure.get_positions()

print("Positions are different:", not np.allclose(base_positions, rattled_positions))
assert not np.allclose(base_positions, rattled_positions)

This UAT has demonstrated that the `PhysicsInformedGenerator` can successfully create a diverse dataset containing a base structure, strained versions, and rattled versions, all from a simple high-level configuration.